# Detrending Injected Light Curves

We inject the synthetic flares into raw long cadence light curves, so they then need to be detrended just as we did with the original light curves. (See notebook [\#1](01_Lightcurve_Detrending.ipynb))

In [1]:
import sys

import numpy as np

from astropy.table import Table, Column

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

sys.path.append('../src')
import detrend

## Long Cadence

Because these light curves already exist in ecsv files, and we don't need to download them using lightkurve, I made a new function `longcad_redetrend_and_score` that does everything that `longcad_detrend_and_score` does, except starting with a light curve table rather than a KID and quarter.

**Note:** This step takes some time to run (~15-30 min). It also will produce some warnings that can safely be ignored.

In [3]:
lc_info_table = Table.read("../data_tables/kepler_longcad_info.ecsv",format="ascii.ecsv")
lc_info_table.add_column(Column(name="injected_score", dtype=float, length=len(lc_info_table)))

In [16]:
for lc_row in lc_info_table:
        
    lc_table = Table.read(lc_row['injected_path'])
    lc_table_new = detrend.longcad_redetrend_and_score(lc_table)
    
    lc_row["injected_score"] = lc_table_new.meta['score']
    lc_table_new.write(lc_row['injected_path'], format="ascii.ecsv", overwrite=True)  

/home/ceb/miniconda3/envs/flares/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [17]:
lc_info_table.write("../data_tables/kepler_longcad_info.ecsv",format="ascii.ecsv", overwrite=True)